# pymongoを使ってタスク管理
## source
[pymongo使い方まとめ](http://nwpct1.hatenablog.com/entry/python-mongodb-pymongo)

## description

```json
{
    'TwitterId': u'kazukousen',
    'TaskNum': 3,
    'Tasks': [
        {
            'TaskId': 0,
            'TaskTitle: u'DeepLearning',
            'TaskDone': False
        },
        {
            'TaskId': 1,
            'TaskTitle: u'pymongo',
            'TaskDone': True,
            'DoneComment': u'たのしかった!',
            'DoneDate': u'2016-05-30'
        },
        {
            'TaskId': 0,
            'TaskTitle: u'Flask',
            'TaskDone': False
        }
    ]
}

...他のユーザ
```

In [ ]:
from pymongo import MongoClient
import datetime as dt

In [ ]:
client = MongoClient()
db = client['UserDB'] # データベースを取得
col = db.User # Userコレクションを取得

## method

- ``find()``: 全部
- ``find({'key': 'value'})``: 条件にマッチするのを全部取得
- ``find_one({'key': 'value'})``: 条件にマッチするのを1つ取ってくる
- ``count()``: findで取得したオブジェクトの数をカウントする

## POST

In [ ]:
# ユーザ登録
col.insert_one({'TwitterId': 'kazukousen', 'TaskNum': 0, 'Tasks': []})

In [ ]:
# ユーザ取得
col.find_one({'TwitterId': 'kazukousen'})

In [ ]:
# タスク登録
task_add = col.find_one({'TwitterId': 'kazukousen'})
task_num = task_add['TaskNum']
task_add['Tasks'].append({'TaskId': task_num, 'TaskTitle': 'DeepLearning', 'TaskDone': False})
task_add['TaskNum'] += 1
col.save(task_add) # DBに反映

## GET

In [ ]:
# ユーザ一覧
for user in col.find():
    print user['TwitterId']

In [ ]:
# あるユーザのタスク一覧
for task in col.find_one({'TwitterId': 'kazukousen'})['Tasks']:
    print task['TaskId'], task['TaskTitle'], task['TaskDone']

## PUT

In [ ]:
# 達成処理(TaskDone:をTrueに変更, DoneComment, DoneDateを追加)
task_id = 0
done_comment = u'最高ですね。'
done_date = dt.datetime.now().strftime("%Y-%m-%d")

user = col.find_one({'TwitterId': 'kazukousen'})
for task in user['Tasks']:
    if task['TaskId'] == task_id:
        task['TaskDone'] = True
        task['DoneComment'] = done_comment
        task['DoneDate'] = done_date
        col.save(user)

In [ ]:
user

## DELETE

In [ ]:
# タスク削除
task_id = 0
user = col.find_one({'TwitterId': 'kazukousen'})
for task in user['Tasks']:
    if task['TaskId'] == task_id:
        user['Tasks'].remove(task)
        user['TaskNum'] -= 1
        col.save(user)
        break
    else:
        print u"そのようなタスクはありません"

In [ ]:
# アカウント削除
account_id = col.find_one({'TwitterId': 'kazukousen'})['_id']
col.delete_one({'_id': account_id})